In [24]:
require 'poisson'

class Modeling
  def initialize(people_num, possibility_correct, lambda_poisson, weight)
    @people_num = people_num
    @people_half = half_num(@people_num) # 終了の値や半分の値に用いる
    @possibility_correct = possibility_correct
    @lambda_poisson = lambda_poisson # ポアソン分布の平均lambda
    @collecting_deadline = lambda_poisson * 2 # 意見収集の締め切り時刻を平均の二倍とする
    @poisson = Poisson.new(@lambda_poisson) # ポアソン分布を扱うクラスのインスタンス
    @weight = weight.to_f # 効用を計算する際の時間に対する重み。精度を重要にしたいなら重みを大きくし、速さを重要にしたいなら重みを小さくする(0.0~1.0の範囲)
  end

  # 人がいつ来るか配列で表す EXAMPLE [1, 0, 1] => 時刻0と時刻2に人がくる
  def simulate_when_people_come(people_num)
    when_people_come = Array.new(@collecting_deadline)
    people_num.times do |t|
      when_people_come[t] = 1
    end
    when_people_come.shuffle
  end


  # 一人目の意見を採用するというアルゴリズム
  def baseline_method_deciding_by_first_person_with_poisson(possibility_correct)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - 重み * かかる時間(決定する時刻 / collecting_deadline)
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ないときは飛ばす
        method_utility += possibility_correct - @weight * (index.to_f / @collecting_deadline)
        break # 一人目の意見しか必要ないのですぐBREAK
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 引数majority_vote_people人で多数決して意見集約を行うというアルゴリズム
  def baseline_method_deciding_by_majority_vote_with_poisson(possibility_correct, majority_vote_people)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - 重み* かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      next if t_people < majority_vote_people # majority_vote_people人集まらない場合は、utility = 0
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count += 1
        if people_count == majority_vote_people
          method_utility +=  (1 - relative_error_by_majority_vote(people_count, possibility_correct)) - @weight * (index.to_f / @collecting_deadline)
          break
        end
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 先にhalf_num(引数temp_people_num)人集まった方の意見を採用するアルゴリズム
  def baseline_method_deciding_by_half_opinion_with_poisson(possibility_correct, temp_people_num)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - 重み * かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      people_count_arr = [] # EXAMPLE [4, 9, 20, 40, 45] 一人目は時刻4にきた　二人目は時刻9にきた...
      next if t_people < temp_people_num # temp_people_num人集まらない場合は、utility = 0 TODO temp_people_num人あつまらなくてもその半数は集まることはあるので厳密ではない
      when_people_come.each_with_index do |elem, index|
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count_arr << index
        people_count += 1
        if people_count == temp_people_num
          relative_error_array = relative_error_array_by_half_opinion(half_num(temp_people_num), possibility_correct)
          expected_error = relative_error_array.inject(:+)
          average_index = 0
          relative_error_array.each_with_index do |e, i| # もっと簡単にかけそう
            average_index += people_count_arr[half_num(temp_people_num) - 1 + i] * (e / expected_error)
          end
          method_utility =  (1 - expected_error) - @weight * (average_index.to_f / @collecting_deadline)
          break
        end
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 誰か一人集まりかつ、時刻引数time_limitで意見収集を打ち切り,多数決で意思決定を行う
  def baseline_method_deciding_by_time_limit_with_poisson(possibility_correct, time_limit)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)  # 人がいつ来るか配列で表す EXAMPLE [1, 0, 1] => 時刻0と時刻2に人がくる
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      when_people_come.each_with_index do |elem, index|
        if index >= time_limit && people_count >= 1
          method_utility = (1 - relative_error_by_majority_vote(people_count, possibility_correct)) - @weight * (index.to_f / @collecting_deadline)
          break
        end
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count += 1
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  def half_num(num)
    (num.to_f / 2).ceil
  end

  # nCk 実行例 5C2 = 5 * 4 / 2! = 10
  def combi(n, k)
    k = n - k if 2 * k > n
    return 1 if k == 0
    ((n - k + 1)..n).reduce(&:*) / (1..k).reduce(&:*)
  end

  # 誤差率をもとめる.先にある割合の意見(finish_num)を集めた方を意見集約の結果とするアルゴリズム
  # このアルゴリズムは必要な人数が動的に変化するので、誤差率をfinish_num個の要素にした配列で返す
  def relative_error_array_by_half_opinion(finish_num, possibility_correct) # possibility_correct => 人が正解する確率p
    relative_error = []
    finish_num.times do |t|
      relative_error << ((1 - possibility_correct)**finish_num) * (possibility_correct**t) * combi(finish_num - 1 + t, t)
    end
    relative_error # EXAMPLE (finish_num = 3 )[0.2, 0.3, 0.5] 0.2は3:0で決まるとき 0.3は3:1で決まるとき 0.5は3:2で決まるとき
  end

  # 誤差率をもとめる.people_num人で多数決を行い意見集約の結果とするアルゴリズム
  def relative_error_by_majority_vote(people_num, possibility_correct) # possibility_correct => 人が正解する確率p
    half_num = half_num(people_num)
    relative_error = 0
    half_num.times do |t|
      relative_error += (possibility_correct**t) * ((1 - possibility_correct)**(people_num - t)) * combi(people_num, t)
    end
    relative_error
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method1(possibility_correct) # possibility_correct => 人が正解する確率p
    # 終了する人数を求める
    finish_num = @people_half
    # 誤差率を求め、返す
    relative_error_array_by_half_opinion(finish_num, possibility_correct).inject(:+)
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method2(possibility_correct)
    # 多数決を行い誤差率を求め、返す
    relative_error_by_majority_vote(@people_num, possibility_correct)
  end

  # 誤差率εを変動させて、必要な人数Xを求める
  def baseline_method3(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 終了する人数を求める
    temp_finish_num = half_num(people_num)
    # 必要な平均人数
    average_num = 0
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      relative_error_array = relative_error_array_by_half_opinion(temp_finish_num, @possibility_correct)
      expected_error = relative_error_array.inject(:+)
      average_num = 0
      relative_error_array.each_with_index do |e, i| # もっと簡単にかけそう
        average_num += e * (i + temp_finish_num) / expected_error
      end
      break if expected_error <= relative_error
      people_num += 2
      temp_finish_num = half_num(people_num)
    end
    # 必要な人数の平均を返す
    average_num
  end

  # 多数決を意見集約のアルゴリズムとする
  def baseline_method4(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      expected_error = relative_error_by_majority_vote(people_num, @possibility_correct)
      break if expected_error <= relative_error
      people_num += 2
    end
    # 必要な人数を返す
    people_num
  end
end


require 'rbplotly'

PEOPELE_NUM = 199
POSSIBILITY_CORRECT = 0.7
LAMBDA_POISSON = 70
WEIGHT = 0.5
DELIMITER = 100
REPEAT = 5

MAJORITY_VOTE_NUM = 5
EARLIER_NUM = 5
TIME_LIMIT = 

# 配列の平均を算出する
def array_average(arr)
  arr.inject(:+) / arr.length
end

# 配列の分散を算出する
def array_variance(arr)
  ave = array_average(arr)
  arr.inject(0) { |a, b| a + (b - ave)**2 } / arr.length
end

model = Modeling.new(PEOPELE_NUM, POSSIBILITY_CORRECT, LAMBDA_POISSON, WEIGHT)

# 縦軸 => 効用utility, 横軸 => 人の正解する確率p
utility_possibility_x_axis = (51..99).to_a

# 引数を変えてアルゴリズムを比較

# 4種類のアルゴリズムを比較する
# 乱数を用いてるのでREPEAT回分の平均をとる
temp_utility_possibility_y_axis_by_first_person = []
temp_utility_possibility_y_axis_by_majority_vote = []
temp_utility_possibility_y_axis_by_half_opinion = []
temp_utility_possibility_y_axis_by_time_limit = []
REPEAT.times do
  temp_utility_possibility_y_axis_by_first_person << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_first_person_with_poisson(e.to_f / DELIMITER) }
  temp_utility_possibility_y_axis_by_majority_vote << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_majority_vote_with_poisson(e.to_f / DELIMITER, MAJORITY_VOTE_NUM) }
  temp_utility_possibility_y_axis_by_half_opinion << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_half_opinion_with_poisson(e.to_f / DELIMITER, EARLIER_NUM) }
  temp_utility_possibility_y_axis_by_time_limit << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_time_limit_with_poisson(e.to_f / DELIMITER, TIME_LIMIT) }
end

# 転置して平均をとる
utility_possibility_y_axis_by_first_person = temp_utility_possibility_y_axis_by_first_person.transpose.map { |e| array_average(e) }
utility_possibility_y_axis_by_majority_vote = temp_utility_possibility_y_axis_by_majority_vote.transpose.map{ |e| array_average(e)}
utility_possibility_y_axis_by_half_opinion = temp_utility_possibility_y_axis_by_half_opinion.transpose.map{ |e| array_average(e)}
utility_possibility_y_axis_by_time_limit = temp_utility_possibility_y_axis_by_time_limit.transpose.map{ |e| array_average(e)}

utility_possibility_by_first_person_trace = [{ x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_first_person }]
utility_possibility_by_majority_vote_trace = [{ x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_majority_vote }]
utility_possibility_by_half_opinion_trace = [{ x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_half_opinion }]
utility_possibility_by_time_limit_trace = [{ x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_time_limit }]

pl = Plotly::Plot.new(data: utility_possibility_by_first_person_trace + utility_possibility_by_majority_vote_trace + utility_possibility_by_half_opinion_trace + utility_possibility_by_time_limit_trace)
pl.layout.xaxis = { title: 'possibility_correct' }
pl.layout.yaxis = { title: 'utility' }
pl.layout.title = "LAMBDA_POISSON = #{LAMBDA_POISSON}, WEIGHT = #{WEIGHT}, MAJORITY_VOTE_NUM = #{MAJORITY_VOTE_NUM}, EARLIER_NUM = #{EARLIER_NUM}, TIME_LIMIT = #{TIME_LIMIT}"
pl.show


(pry):6307: warning: already initialized constant PEOPELE_NUM
(pry):6041: warning: previous definition of PEOPELE_NUM was here
(pry):6308: warning: already initialized constant POSSIBILITY_CORRECT
(pry):6042: warning: previous definition of POSSIBILITY_CORRECT was here
(pry):6309: warning: already initialized constant LAMBDA_POISSON
(pry):6043: warning: previous definition of LAMBDA_POISSON was here
(pry):6310: warning: already initialized constant WEIGHT
(pry):6044: warning: previous definition of WEIGHT was here
(pry):6311: warning: already initialized constant DELIMITER
(pry):6045: warning: previous definition of DELIMITER was here
(pry):6312: warning: already initialized constant REPEAT
(pry):6046: warning: previous definition of REPEAT was here
(pry):6314: warning: already initialized constant MAJORITY_VOTE_NUM
(pry):6048: warning: previous definition of MAJORITY_VOTE_NUM was here
(pry):6315: warning: already initialized constant EARLIER_NUM
(pry):6049: warning: previous definitio

#<Plotly::Offline::HTML:0x00007faf4622be90 @id="f66580a5-4616-4c0d-baa5-c70df8e19d85", @data=[{:x=>[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], :y=>[0.5063097835693411, 0.5161410639493804, 0.5262102502006807, 0.5369773697639991, 0.5465257318405932, 0.5566941243226649, 0.5668766298816998, 0.5762741164305918, 0.5872106172000675, 0.5954563362551573, 0.6062224175802567, 0.6161748541040311, 0.6261110491968884, 0.6364510358851762, 0.6462019263510873, 0.6563655706157344, 0.6667894161630284, 0.6759974356412112, 0.6866013624154002, 0.6967768598613657, 0.7056238646116347, 0.7160078311367254, 0.7264207948720578, 0.7369867891078792, 0.7463368761690595, 0.7562441187714786, 0.7664331461669015, 0.7771043992772257, 0.7865490917030445, 0.7964671395231987, 0.806037130418858, 0.8168498569410032, 0.8267404405769027, 0.8367045538524873, 0.8463694854197359, 0.8560729096547796, 0.8663683615076726, 0.8762258645503975, 0.8858642608305143, 0.8961333969299556, 0.9063828662514297, 0.9160304954013732, 0.9255526697223144, 0.9366147659839431, 0.9462066774410631, 0.9562878603596555, 0.9660732288677945, 0.9773983154072685, 0.9859572495283434]}, {:x=>[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], :y=>[0.48555735132345107, 0.5051518262267147, 0.5247295861538503, 0.542566277861769, 0.5602886562406815, 0.5793606215756931, 0.5966865542818278, 0.6157072508851711, 0.6320898872537953, 0.6485427024909626, 0.6662327745977277, 0.6840081791318889, 0.7020695049145254, 0.7181745892547087, 0.7317952359622554, 0.7471533887090068, 0.762815293463005, 0.7756952956390778, 0.7918862948387929, 0.8034931618821155, 0.8186680549726482, 0.8296729911850129, 0.8423187719021097, 0.8534897826694415, 0.8641044340068378, 0.8731128661617988, 0.8842084741743479, 0.8937825414772582, 0.9013798271667566, 0.9097992525558045, 0.9184383149957751, 0.9250724686942838, 0.9285134275988922, 0.9360887375774574, 0.9408185988733472, 0.9465009699101377, 0.9505433782637882, 0.9538110341553148, 0.9578767204656099, 0.9593055413258724, 0.9616947731752015, 0.9639389776561981, 0.9657786779343638, 0.9650641141589498, 0.9671451750857185, 0.9685523425300511, 0.9675641067084187, 0.9686528472928314, 0.9687834223039677]}, {:x=>[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], :y=>[0.49276439034037034, 0.5115511423906666, 0.5301373303825988, 0.5471668290983778, 0.5673456332996897, 0.5839841410184745, 0.6024711029707797, 0.619641784688513, 0.6389602028653332, 0.6553085369135558, 0.6727218129986672, 0.688790937264199, 0.7072282793501409, 0.7211357429237852, 0.737179178573791, 0.7523240335360792, 0.7671099278774685, 0.7823822348281139, 0.7948183008328132, 0.8094262454386181, 0.8213937014542514, 0.835021724169934, 0.8464542806274468, 0.8591436518777462, 0.8690254191438059, 0.8787775138165623, 0.8890320592431294, 0.8972071276137891, 0.9072608300496426, 0.913705926375979, 0.9219620834656359, 0.9277364282915805, 0.9342286827832451, 0.940001995625791, 0.9451217245600795, 0.948695068329096, 0.9533729411959765, 0.9580281709215892, 0.9606792427079002, 0.9635034567648342, 0.9646606552632067, 0.9682240328843656, 0.9672662225657709, 0.9689447054632071, 0.9697919779341506, 0.9697124139503902, 0.970275813499007, 0.9719111215097455, 0.9724914600156591]}, {:x=>[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], :y=>[0.6056704530284549, 0.6224348798765859, 0.6346102968935975, 0.6388943521472015, 0.6611180146476439, 0.6655105684944865, 0.667469109896852

#<CZTop::Socket::PUB:0x7faf47111b00 last_endpoint="tcp://127.0.0.1:55147">